In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Cinema_Audience_Forecasting_challenge/movie_theater_id_relation/movie_theater_id_relation.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/date_info/date_info.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/sample_submission/sample_submission.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_theaters/booknow_theaters.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/cinePOS_booking/cinePOS_booking.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/cinePOS_theaters/cinePOS_theaters.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_visits/booknow_visits.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_booking/booknow_booking.csv


In [2]:
cinepos_theaters  = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/cinePOS_theaters/cinePOS_theaters.csv')
booknow_theaters  = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_theaters/booknow_theaters.csv')
theater_relation  = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/movie_theater_id_relation/movie_theater_id_relation.csv')
cinepos_booking   = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/cinePOS_booking/cinePOS_booking.csv')
booknow_booking   = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_booking/booknow_booking.csv')
booknow_visits    = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_visits/booknow_visits.csv')
date_info         = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/date_info/date_info.csv')
sample_submission = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/sample_submission/sample_submission.csv')

# **EDA for cinepos_theaters**

In [3]:
cinepos_theaters.shape

(4690, 5)

In [4]:
cinepos_theaters.head()

,cine_theater_id,theater_type,theater_area,latitude,longitude
0,cinePOS_05466,Other,Area_104,22.619233,78.113017
1,cinePOS_08708,Other,Area_104,22.619233,78.113017
2,cinePOS_00753,Other,Area_104,22.619233,78.113017
3,cinePOS_05776,Other,Area_104,22.619233,78.113017
4,cinePOS_07536,Other,Area_104,23.004410,79.934515


In [5]:
cinepos_theaters.isnull().sum()

cine_theater_id       0
theater_type          0
theater_area          0
latitude           3861
longitude          3861
dtype: int64

In [6]:
cinepos_theaters.isnull().sum()/4690*100

cine_theater_id     0.000000
theater_type        0.000000
theater_area        0.000000
latitude           82.324094
longitude          82.324094
dtype: float64

In [7]:
cinepos_theaters.nunique()

cine_theater_id    4690
theater_type          3
theater_area        119
latitude            108
longitude           108
dtype: int64

In [8]:
cinepos_theaters_cleaned = cinepos_theaters.copy()
cinepos_theaters_cleaned.drop(columns=['latitude', 'longitude'], inplace=True)

In [9]:
cinepos_theaters_cleaned.head()

,cine_theater_id,theater_type,theater_area
0,cinePOS_05466,Other,Area_104
1,cinePOS_08708,Other,Area_104
2,cinePOS_00753,Other,Area_104
3,cinePOS_05776,Other,Area_104
4,cinePOS_07536,Other,Area_104


# **EDA for booknow_theaters**

In [10]:
booknow_theaters.shape

(829, 5)

In [11]:
booknow_theaters.head()

,book_theater_id,theater_type,theater_area,latitude,longitude
0,book_00093,Drama,Area_001,22.619233,78.113017
1,book_00078,Drama,Area_001,22.619233,78.113017
2,book_00291,Drama,Area_001,22.619233,78.113017
3,book_00258,Drama,Area_001,22.619233,78.113017
4,book_00212,Drama,Area_002,23.004410,79.934515


In [12]:
booknow_theaters.isnull().sum()

book_theater_id    515
theater_type         0
theater_area         0
latitude             0
longitude            0
dtype: int64

In [13]:
booknow_theaters.isnull().sum()/829 *100

book_theater_id    62.12304
theater_type        0.00000
theater_area        0.00000
latitude            0.00000
longitude           0.00000
dtype: float64

In [14]:
booknow_theaters.nunique()

book_theater_id    314
theater_type         4
theater_area       103
latitude           108
longitude          108
dtype: int64

In [15]:
booknow_theaters_cleaned = booknow_theaters.copy()
booknow_theaters_cleaned.shape

(829, 5)

In [16]:
booknow_theaters_cleaned.dropna(subset=['book_theater_id'], inplace=True)

In [17]:
booknow_theaters_cleaned.shape

(314, 5)

# **EDA for theater_relation**

In [18]:
theater_relation.shape

(150, 2)

In [19]:
theater_relation.head()

,book_theater_id,cine_theater_id
0,book_00509,cinePOS_01261
1,book_00063,cinePOS_02467
2,book_00054,cinePOS_08923
3,book_00094,cinePOS_02479
4,book_00052,cinePOS_06750


In [20]:
theater_relation.isnull().sum()

book_theater_id    0
cine_theater_id    0
dtype: int64

In [21]:
theater_relation.nunique()

book_theater_id    150
cine_theater_id    150
dtype: int64

In [22]:
theater_relation_cleaned = theater_relation.copy()

# **EDA for cinepos_booking**

In [23]:
cinepos_booking.shape

(1641966, 4)

In [24]:
cinepos_booking.head()

,cine_theater_id,show_datetime,booking_datetime,tickets_sold
0,cinePOS_00001,2023-01-01 11:00:00,2023-01-01 09:00:00,1
1,cinePOS_00002,2023-01-01 13:00:00,2023-01-01 06:00:00,3
2,cinePOS_00003,2023-01-01 16:00:00,2023-01-01 14:00:00,2
3,cinePOS_00004,2023-01-01 17:00:00,2023-01-01 11:00:00,5
4,cinePOS_00005,2023-01-01 17:00:00,2023-01-01 03:00:00,13


In [25]:
cinepos_booking.isnull().sum()

cine_theater_id     0
show_datetime       0
booking_datetime    0
tickets_sold        0
dtype: int64

In [26]:
cinepos_booking.nunique()

cine_theater_id     13161
show_datetime        8283
booking_datetime    10152
tickets_sold           97
dtype: int64

In [27]:
cinepos_booking_cleaned = cinepos_booking.copy()

# **EDA for booknow_booking**

In [28]:
booknow_booking.shape

(68336, 4)

In [29]:
booknow_booking.head()

,book_theater_id,show_datetime,booking_datetime,tickets_booked
0,book_00244,2023-01-01 19:00:00,2023-01-01 16:00:00,1
1,book_00740,2023-01-01 19:00:00,2023-01-01 19:00:00,3
2,book_00740,2023-01-01 19:00:00,2023-01-01 19:00:00,6
3,book_00244,2023-01-01 20:00:00,2023-01-01 16:00:00,2
4,book_00151,2023-01-01 20:00:00,2023-01-01 01:00:00,5


In [30]:
booknow_booking.isnull().sum()

book_theater_id     0
show_datetime       0
booking_datetime    0
tickets_booked      0
dtype: int64

In [31]:
booknow_booking.nunique()

book_theater_id      301
show_datetime       3866
booking_datetime    6351
tickets_booked        66
dtype: int64

In [32]:
booknow_booking_cleaned = booknow_booking.copy()

# **EDA for booknow_visits**

In [33]:
booknow_visits.shape


(214046, 3)

In [34]:
booknow_visits.head()

,book_theater_id,show_date,audience_count
0,book_00001,2023-01-13,50
1,book_00001,2023-01-14,64
2,book_00001,2023-01-15,58
3,book_00001,2023-01-16,44
4,book_00001,2023-01-18,12


In [35]:
booknow_visits.isnull().sum()


book_theater_id    0
show_date          0
audience_count     0
dtype: int64

In [36]:
booknow_visits.nunique()

book_theater_id    826
show_date          424
audience_count     191
dtype: int64

In [37]:
booknow_visits_cleaned = booknow_visits.copy()

# **EDA for date_info**

In [38]:
date_info.shape

(547, 2)

In [39]:
date_info.head()

,show_date,day_of_week
0,2023-01-01,Sunday
1,2023-01-02,Monday
2,2023-01-03,Tuesday
3,2023-01-04,Wednesday
4,2023-01-05,Thursday


In [40]:
date_info.isnull().sum()

show_date      0
day_of_week    0
dtype: int64

In [41]:
date_info.nunique()

show_date      547
day_of_week      7
dtype: int64

In [42]:
date_info_cleaned = date_info.copy()

# **EDA for sample_submission**

In [43]:
sample_submission.shape

(38062, 2)

In [44]:
sample_submission.head()

,ID,audience_count
0,book_00001_2024-03-01,0
1,book_00001_2024-03-02,0
2,book_00001_2024-03-03,0
3,book_00001_2024-03-04,0
4,book_00001_2024-03-06,0


In [45]:
sample_submission.isnull().sum()

ID                0
audience_count    0
dtype: int64

In [46]:
sample_submission.nunique()

ID                38062
audience_count        1
dtype: int64

# **Merging Datasets**

In [47]:
master_df = booknow_visits_cleaned.copy()

In [48]:
master_df.shape

(214046, 3)

In [49]:
master_df = pd.merge(master_df, date_info_cleaned, on='show_date', how='left')

In [50]:
master_df.shape

(214046, 4)

In [51]:
master_df = pd.merge(master_df, booknow_theaters_cleaned, on='book_theater_id', how='left')

In [52]:
master_df.shape

(214046, 8)

In [53]:
master_df.head()

,book_theater_id,show_date,audience_count,day_of_week,theater_type,theater_area,latitude,longitude
0,book_00001,2023-01-13,50,Friday,Drama,Area_002,23.00441,79.934515
1,book_00001,2023-01-14,64,Saturday,Drama,Area_002,23.00441,79.934515
2,book_00001,2023-01-15,58,Sunday,Drama,Area_002,23.00441,79.934515
3,book_00001,2023-01-16,44,Monday,Drama,Area_002,23.00441,79.934515
4,book_00001,2023-01-18,12,Wednesday,Drama,Area_002,23.00441,79.934515


In [54]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214046 entries, 0 to 214045
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   book_theater_id  214046 non-null  object 
 1   show_date        214046 non-null  object 
 2   audience_count   214046 non-null  int64  
 3   day_of_week      214046 non-null  object 
 4   theater_type     80150 non-null   object 
 5   theater_area     80150 non-null   object 
 6   latitude         80150 non-null   float64
 7   longitude        80150 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 13.1+ MB


In [55]:
master_df.isnull().sum()

book_theater_id         0
show_date               0
audience_count          0
day_of_week             0
theater_type       133896
theater_area       133896
latitude           133896
longitude          133896
dtype: int64

In [56]:
master_df.isnull().sum()/214046 *100

book_theater_id     0.000000
show_date           0.000000
audience_count      0.000000
day_of_week         0.000000
theater_type       62.554778
theater_area       62.554778
latitude           62.554778
longitude          62.554778
dtype: float64

In [57]:
master_df['is_theater_info_missing'] = master_df['theater_type'].isnull().astype(int)

In [58]:
cat_cols= ['theater_type','theater_area']

In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


ct=ColumnTransformer([
    ('cat',SimpleImputer(strategy='constant',fill_value='Unknown'),cat_cols)
],remainder='passthrough',verbose_feature_names_out=False).set_output(transform='pandas')


In [60]:
master_df=ct.fit_transform(master_df)

In [61]:
master_df=master_df.drop(columns=['latitude', 'longitude'])


In [62]:
master_df['is_theater_info_missing'].unique()

array([0, 1])

# **Feature Engineering**

In [63]:
master_df['show_date'] = pd.to_datetime(master_df['show_date'])

master_df['year'] = master_df['show_date'].dt.year
master_df['month'] = master_df['show_date'].dt.month
master_df['day'] = master_df['show_date'].dt.day
master_df['week_of_year'] = master_df['show_date'].dt.isocalendar().week.astype(int)
master_df['is_weekend'] = (master_df['day_of_week'].isin(['Saturday', 'Sunday'])).astype(int)

In [64]:
book_bookings = booknow_booking_cleaned.copy()
book_bookings['show_datetime'] = pd.to_datetime(book_bookings['show_datetime'])
book_bookings['booking_datetime'] = pd.to_datetime(book_bookings['booking_datetime'])
book_bookings['show_date'] = pd.to_datetime(book_bookings['show_datetime'].dt.date)
book_bookings['booking_lead_days'] = (book_bookings['show_datetime'] - book_bookings['booking_datetime']).dt.total_seconds() / (24 * 60 * 60)

online_features = book_bookings.groupby(['book_theater_id', 'show_date']).agg(
    total_online_tickets=('tickets_booked', 'sum'),
    avg_online_lead_days=('booking_lead_days', 'mean')
).reset_index()

master_df = pd.merge(master_df, online_features, on=['book_theater_id', 'show_date'], how='left')

In [65]:
master_df.tail()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,theater_type,theater_area,book_theater_id,show_date,audience_count,day_of_week,is_theater_info_missing,year,month,day,week_of_year,is_weekend,total_online_tickets,avg_online_lead_days
214041,Unknown,Unknown,book_00829,2024-02-23,16,Friday,1,2024,2,23,8,0,NaN,NaN
214042,Unknown,Unknown,book_00829,2024-02-24,8,Saturday,1,2024,2,24,8,1,NaN,NaN
214043,Unknown,Unknown,book_00829,2024-02-25,16,Sunday,1,2024,2,25,8,1,NaN,NaN
214044,Unknown,Unknown,book_00829,2024-02-27,16,Tuesday,1,2024,2,27,9,0,NaN,NaN
214045,Unknown,Unknown,book_00829,2024-02-28,16,Wednesday,1,2024,2,28,9,0,NaN,NaN


In [66]:
pos_bookings = cinepos_booking_cleaned.copy()
pos_bookings_linked = pd.merge(pos_bookings, theater_relation_cleaned, on='cine_theater_id', how='inner')
pos_bookings_linked['show_datetime'] = pd.to_datetime(pos_bookings_linked['show_datetime'])
pos_bookings_linked['show_date'] = pd.to_datetime(pos_bookings_linked['show_datetime'].dt.date)

pos_features = pos_bookings_linked.groupby(['book_theater_id', 'show_date']).agg(
    total_pos_tickets=('tickets_sold', 'sum')
).reset_index()

master_df = pd.merge(master_df, pos_features, on=['book_theater_id', 'show_date'], how='left')

In [67]:
master_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,theater_type,theater_area,book_theater_id,show_date,audience_count,day_of_week,is_theater_info_missing,year,month,day,week_of_year,is_weekend,total_online_tickets,avg_online_lead_days,total_pos_tickets
0,Drama,Area_002,book_00001,2023-01-13,50,Friday,0,2023,1,13,2,0,NaN,NaN,NaN
1,Drama,Area_002,book_00001,2023-01-14,64,Saturday,0,2023,1,14,2,1,NaN,NaN,NaN
2,Drama,Area_002,book_00001,2023-01-15,58,Sunday,0,2023,1,15,2,1,NaN,NaN,NaN
3,Drama,Area_002,book_00001,2023-01-16,44,Monday,0,2023,1,16,3,0,NaN,NaN,NaN
4,Drama,Area_002,book_00001,2023-01-18,12,Wednesday,0,2023,1,18,3,0,NaN,NaN,NaN


In [68]:
feature_cols = ['total_online_tickets', 'avg_online_lead_days', 'total_pos_tickets']

In [69]:
ctf=ColumnTransformer([
    ('fea',SimpleImputer(strategy='constant',fill_value=0),feature_cols)
],remainder='passthrough',verbose_feature_names_out=False).set_output(transform='pandas')

In [70]:
master_df=ctf.fit_transform(master_df)

In [71]:
master_df.drop(columns=['show_date'], inplace=True)

In [72]:
master_df.head()

,total_online_tickets,avg_online_lead_days,total_pos_tickets,theater_type,theater_area,book_theater_id,audience_count,day_of_week,is_theater_info_missing,year,month,day,week_of_year,is_weekend
0,0.0,0.0,0.0,Drama,Area_002,book_00001,50,Friday,0,2023,1,13,2,0
1,0.0,0.0,0.0,Drama,Area_002,book_00001,64,Saturday,0,2023,1,14,2,1
2,0.0,0.0,0.0,Drama,Area_002,book_00001,58,Sunday,0,2023,1,15,2,1
3,0.0,0.0,0.0,Drama,Area_002,book_00001,44,Monday,0,2023,1,16,3,0
4,0.0,0.0,0.0,Drama,Area_002,book_00001,12,Wednesday,0,2023,1,18,3,0


# **Spliting**

In [73]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score 
import numpy as np

import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [74]:
train_val_df = master_df.copy()


In [75]:
categorical_cols = train_val_df.select_dtypes(include=['object']).columns.tolist()

In [76]:
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_val_df[col] = le.fit_transform(train_val_df[col])
    encoders[col] = le

In [77]:
train_val_df.head()

,total_online_tickets,avg_online_lead_days,total_pos_tickets,theater_type,theater_area,book_theater_id,audience_count,day_of_week,is_theater_info_missing,year,month,day,week_of_year,is_weekend
0,0.0,0.0,0.0,2,1,0,50,0,0,2023,1,13,2,0
1,0.0,0.0,0.0,2,1,0,64,2,0,2023,1,14,2,1
2,0.0,0.0,0.0,2,1,0,58,3,0,2023,1,15,2,1
3,0.0,0.0,0.0,2,1,0,44,1,0,2023,1,16,3,0
4,0.0,0.0,0.0,2,1,0,12,6,0,2023,1,18,3,0


In [78]:
train_val_df['temp_date'] = pd.to_datetime(train_val_df[['year', 'month', 'day']])
train_df = train_val_df[train_val_df['temp_date'] < '2024-02-01']
val_df = train_val_df[train_val_df['temp_date'] >= '2024-02-01']
train_df = train_df.drop(columns=['temp_date'])
val_df = val_df.drop(columns=['temp_date'])
features = [col for col in train_df.columns if col not in ['audience_count']]

In [79]:
X_train = train_df[features]
y_train = train_df['audience_count']
X_val = val_df[features]
y_val = val_df['audience_count']

In [80]:
X_train.shape

(194235, 13)

In [81]:
X_val.shape

(19811, 13)

# **For Submission File** 

In [82]:
test_df = sample_submission.copy()

In [83]:
test_df['book_theater_id'] = test_df['ID'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])
test_df['show_date'] = pd.to_datetime(test_df['ID'].apply(lambda x: x.split('_')[2]))

In [84]:
test_df.drop(columns=['audience_count'], inplace=True)

In [85]:
date_info_cleaned['show_date'] = pd.to_datetime(date_info_cleaned['show_date'])
test_df = pd.merge(test_df, date_info_cleaned, on='show_date', how='left')

In [86]:
test_df['year'] = test_df['show_date'].dt.year
test_df['month'] = test_df['show_date'].dt.month
test_df['day'] = test_df['show_date'].dt.day
test_df['week_of_year'] = test_df['show_date'].dt.isocalendar().week.astype(int)
test_df['is_weekend'] = (test_df['day_of_week'].isin(['Saturday', 'Sunday'])).astype(int)
test_df = pd.merge(test_df, booknow_theaters_cleaned, on='book_theater_id', how='left')
test_df['is_theater_info_missing'] = test_df['theater_type'].isnull().astype(int)


In [87]:
test_df=ct.fit_transform(test_df)

In [88]:
online_features['show_date'] = pd.to_datetime(online_features['show_date'])
pos_features['show_date'] = pd.to_datetime(pos_features['show_date'])
test_df = pd.merge(test_df, online_features, on=['book_theater_id', 'show_date'], how='left')
test_df = pd.merge(test_df, pos_features, on=['book_theater_id', 'show_date'], how='left')

In [89]:
test_df=ctf.fit_transform(test_df)

In [90]:
test_df.head()

,total_online_tickets,avg_online_lead_days,total_pos_tickets,theater_type,theater_area,ID,book_theater_id,show_date,day_of_week,year,month,day,week_of_year,is_weekend,latitude,longitude,is_theater_info_missing
0,0.0,0.0,0.0,Drama,Area_002,book_00001_2024-03-01,book_00001,2024-03-01,Friday,2024,3,1,9,0,23.00441,79.934515,0
1,0.0,0.0,0.0,Drama,Area_002,book_00001_2024-03-02,book_00001,2024-03-02,Saturday,2024,3,2,9,1,23.00441,79.934515,0
2,0.0,0.0,0.0,Drama,Area_002,book_00001_2024-03-03,book_00001,2024-03-03,Sunday,2024,3,3,9,1,23.00441,79.934515,0
3,0.0,0.0,0.0,Drama,Area_002,book_00001_2024-03-04,book_00001,2024-03-04,Monday,2024,3,4,10,0,23.00441,79.934515,0
4,0.0,0.0,0.0,Drama,Area_002,book_00001_2024-03-06,book_00001,2024-03-06,Wednesday,2024,3,6,10,0,23.00441,79.934515,0


In [91]:
for col, le in encoders.items():
    test_df[col] = test_df[col].map(lambda s: str(s) if s not in le.classes_ else s)
    new_labels = set(test_df[col]) - set(le.classes_)
    if new_labels:
        le.classes_ = np.append(le.classes_, sorted(list(new_labels)))
    test_df[col] = le.transform(test_df[col])
X_test = test_df[features]
X_test.shape

(38062, 13)

In [92]:
models = {
    "LightGBM": lgb.LGBMRegressor(random_state=42),
    "XGBoost": xgb.XGBRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=100)
}

In [93]:
results_r2 = {}
for name, model in models.items():
    print(f"{name}")
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    score = r2_score(y_val, predictions)
    results_r2[name] = score
    print(f"{name}:{score:.4f}")

LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010049 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 849
[LightGBM] [Info] Number of data points in the train set: 194235, number of used features: 13
[LightGBM] [Info] Start training from score 41.727474
LightGBM:0.3031
XGBoost
XGBoost:0.3615
Random Forest
Random Forest:0.5282


In [94]:
def create_basic_features(df, date_info, theater_info):
    df_copy = df.copy()
    df_copy['show_date'] = pd.to_datetime(df_copy['show_date'])
    df_copy = pd.merge(df_copy, date_info, on='show_date', how='left')
    df_copy['year'] = df_copy['show_date'].dt.year
    df_copy['month'] = df_copy['show_date'].dt.month
    df_copy['day'] = df_copy['show_date'].dt.day
    df_copy['week_of_year'] = df_copy['show_date'].dt.isocalendar().week.astype(int)
    df_copy['day_of_week'] = df_copy['show_date'].dt.day_name()
    df_copy['is_weekend'] = (df_copy['show_date'].dt.dayofweek >= 5).astype(int)
    df_copy = pd.merge(df_copy, theater_info, on='book_theater_id', how='left')
    df_copy['is_theater_info_missing'] = df_copy['theater_type'].isnull().astype(int)
    df_copy['theater_type'] = df_copy['theater_type'].fillna('Unknown')
    df_copy['theater_area'] = df_copy['theater_area'].fillna('Unknown')
    return df_copy

In [95]:
full_df = master_df.copy()
full_df['show_date'] = pd.to_datetime(full_df[['year', 'month', 'day']])

In [96]:
full_df = full_df.sort_values(by=['book_theater_id', 'show_date']).reset_index(drop=True)
lags = [7, 14, 28]
for lag in lags:
    full_df[f'audience_lag_{lag}'] = full_df.groupby('book_theater_id')['audience_count'].shift(lag)
windows = [7, 14]
for w in windows:
    full_df[f'audience_roll_mean_{w}'] = full_df.groupby('book_theater_id')['audience_count'].shift(1).rolling(window=w, min_periods=1).mean()
full_df.fillna(0, inplace=True)

In [97]:
last_date = full_df['show_date'].max()
validation_start_date = last_date - pd.DateOffset(months=2)
train_df = full_df[full_df['show_date'] < validation_start_date]
val_df = full_df[full_df['show_date'] >= validation_start_date]
train_df = train_df.drop(columns=['show_date'])
val_df = val_df.drop(columns=['show_date'])

In [98]:
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    val_df[col] = val_df[col].map(lambda s: s if s in le.classes_ else 'Unknown')
    le_classes = le.classes_.tolist()
    if 'Unknown' not in le_classes:
        le.classes_ = np.append(le.classes_, 'Unknown')
    val_df[col] = le.transform(val_df[col])
    encoders[col] = le
features = [col for col in train_df.columns if col not in ['audience_count']]

In [99]:
X_train = train_df[features]
y_train = train_df['audience_count']
X_val = val_df[features]
y_val = val_df['audience_count']
X_full_train = pd.concat([X_train, X_val])
y_full_train = pd.concat([y_train, y_val])

In [100]:
test_df = sample_submission[['ID']].copy()
test_df['book_theater_id'] = test_df['ID'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])
test_df['show_date'] = pd.to_datetime(test_df['ID'].apply(lambda x: x.split('_')[2]))


In [101]:
test_features = create_basic_features(test_df, date_info_cleaned, booknow_theaters_cleaned)

In [102]:
test_features = test_features.merge(online_features, on=['book_theater_id', 'show_date'], how='left')
test_features = test_features.merge(pos_features, on=['book_theater_id', 'show_date'], how='left')

In [103]:
for lag in lags:
    last_values = full_df.groupby('book_theater_id')[f'audience_lag_{lag}'].last().rename(f'audience_lag_{lag}')
    test_features = test_features.merge(last_values, on='book_theater_id', how='left')

for w in windows:
    last_values = full_df.groupby('book_theater_id')[f'audience_roll_mean_{w}'].last().rename(f'audience_roll_mean_{w}')
    test_features = test_features.merge(last_values, on='book_theater_id', how='left')

test_features.fillna(0, inplace=True)

In [104]:
for col, le in encoders.items():
    test_features[col] = test_features[col].map(lambda s: str(s) if s not in le.classes_ else s)
    new_labels = set(test_features[col]) - set(le.classes_)
    if new_labels:
        le.classes_ = np.append(le.classes_, sorted(list(new_labels)))
    test_features[col] = le.transform(test_features[col])

In [105]:
X_test = test_features[features]
X_test.shape

(38062, 18)

In [106]:
model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1, max_depth=20)

In [107]:
model.fit(X_train, y_train)
val_predictions = model.predict(X_val)
val_r2 = r2_score(y_val, val_predictions)

In [108]:
val_r2

0.48355541732609353

In [109]:
model.fit(X_full_train, y_full_train)

final_predictions = model.predict(X_test)
final_predictions[final_predictions < 0] = 0
final_predictions = np.round(final_predictions).astype(int)
submission_df = test_features[['ID']].copy()
submission_df['audience_count'] = final_predictions
submission_df.to_csv('submission.csv', index=False)

# **For Registration**

In [110]:
# import pandas as pd
# df = pd.read_csv("/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_visits/booknow_visits.csv")
# mean_audience = df['audience_count'].mean()
# print(f"Calculated Mean Audience Count: {mean_audience}")
# submission_df = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/sample_submission/sample_submission.csv')
# submission_df['audience_count'] = int(round(mean_audience, 0))
# submission_df.to_csv('submission.csv', index=False)
# print("Done!!!!!!")
# print(submission_df.head())